In [1]:
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
from matplotlib import rcParams
from matplotlib import animation
from IPython.display import HTML
import matplotlib.cm as cm 
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [34]:
def energy_at_site(sp,alpha,sigma,sigma1,ix,iy):
    """ Computes the contribution to the energy for a given spin
    at location ix,iy for a lattice with periodic boundary conditions
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    sigma   : int
        spin at site ix,iy
    ix   : int
        location in x
    iy   : int
        location in y
    
   
    Returns:
    -------
    energy: energy for the given configuration
    """
    
    #bordure composé de spin up uniquement
    if ix==0 :
        energy=-alpha*sigma1*(sp[(ix+1),iy]+sp[ix,(iy-1)]+sp[ix,(iy+1)])
    
    elif ix==nx-1 :
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[ix,(iy-1)]+sp[ix,(iy+1)])
    
    elif iy==0 :
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[(ix+1),iy]+sp[ix,(iy+1)])
    
    elif iy==ny-1 :
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[(ix+1),iy]+sp[ix,(iy-1)])
    #coins    
    elif ix==0 and iy==0:
        energy=-alpha*sigma1*(sp[(ix+1),iy]+sp[ix,(iy+1)])
    
    elif ix==nx-1 and iy==0:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[ix,(iy+1)])
    
    elif iy==ny-1 and ix==0:
        energy=-alpha*sigma1*(sp[(ix+1),iy]+sp[ix,(iy-1)])
    
    elif iy==ny-1 and ix==nx-1:
        energy=-alpha*sigma1*(sp[(ix-1),iy]+sp[ix,(iy-1)])
    #carré intérieur
    else: 
        energy=-alpha*sigma*(sp[(ix-1),iy]+sp[(ix+1),iy]+sp[ix,(iy-1)]+sp[ix,(iy+1)])
        
    return energy

In [35]:
def metropolis_at_site(sp,sigma1,alpha,ix,iy):
    """ Flips a dipole at site ix, iy when probability condition is met 
   
    Parameters:
    ----------
    sp: numpy array
        array of spins
    alpha  : real
        coupling constant J/(kb*T)
    ix   : int
        location in x
    iy   : int
        location in y
    """
    

    sigma=sp[ix,iy]
    energy_before_flip = energy_at_site(sp,alpha,sigma,sigma1,ix,iy)
    sigma = -sigma
    energy_if_site_flipped = energy_at_site(sp,alpha,sigma,sigma1,ix,iy)
    
    # Flip the site with Metropolis probability
    # Condition is always satisifed if dE < 0
    if (np.random.random_sample()<np.exp(-(energy_if_site_flipped \
                                               -energy_before_flip))):
        if ix!=0:
            if iy!=0:
                if ix!=nx-1:
                    if iy!=ny-1:
                        sp[ix,iy]=-sp[ix,iy]
        #print(sp)

In [52]:
def ising_model_metropolis(sp,sigma1, NMC, nx, ny, alpha):
    """ Creates a sequence of states for the Ising model using
    the Metropolis algorithm
   
    Parameters:
    ----------
    sp   : initial lattice state
    nx   : int
        Discretization points in x
    ny   : int
        Discretization points in y
    NMC  : int
        Number of states to create
    alpha  : real
        coupling constant J/(kb*T)
    Returns:
    -------
    states: sequence of states
    """
    states = np.empty([NMC+1,nx,ny])
    states[0] = sp.copy()
    
    for i in range(1,NMC+1):
        for j in range(1,(nx-1)*(ny-1)):
            ix=np.random.randint(1,nx-1)
            iy=np.random.randint(1,ny-1)
            metropolis_at_site(sp,sigma1,alpha,ix,iy)
        states[i]=sp.copy()
    return states

In [53]:
sigma1=1
sigma=-1
nx=4
ny=4
ix=1
iy=1
sp=np.ones([ny,nx])
NMC=100
alpha=0.05

In [54]:
E=energy_at_site(sp,alpha,sigma,sigma1,ix,iy)
I=ising_model_metropolis(sp,sigma1, NMC, nx, ny, alpha)
print(I)

[[[ 1.  1.  1.  1.]
  [ 1.  1.  1.  1.]
  [ 1.  1.  1.  1.]
  [ 1.  1.  1.  1.]]

 [[ 1.  1.  1.  1.]
  [ 1.  1. -1.  1.]
  [ 1.  1.  1.  1.]
  [ 1.  1.  1.  1.]]

 [[ 1.  1.  1.  1.]
  [ 1. -1. -1.  1.]
  [ 1. -1. -1.  1.]
  [ 1.  1.  1.  1.]]

 ...

 [[ 1.  1.  1.  1.]
  [ 1. -1.  1.  1.]
  [ 1. -1. -1.  1.]
  [ 1.  1.  1.  1.]]

 [[ 1.  1.  1.  1.]
  [ 1. -1.  1.  1.]
  [ 1.  1.  1.  1.]
  [ 1.  1.  1.  1.]]

 [[ 1.  1.  1.  1.]
  [ 1.  1.  1.  1.]
  [ 1. -1.  1.  1.]
  [ 1.  1.  1.  1.]]]
